# Migrate Old 3-Digit Cluster IDs to New Hash-Based IDs

This notebook migrates old 3-digit cluster IDs in the mmap directory to new hash-based cluster IDs generated from geometry and country code.

In [9]:
import sys
from pathlib import Path
import json
import hashlib
import shutil

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from src.data.database import DownloadDatabase
from src.data.config import Config

In [13]:
# Initialize config and database
config = Config()
mmap_path = config.DATA_DIR / "landsat_mmap"

print(f"Database: {config.DB_PATH}")
print(f"MMAP path: {mmap_path}")

# Connect directly to avoid schema creation issues with old databases
import sqlite3
conn = sqlite3.connect(str(config.DB_PATH))
conn.row_factory = sqlite3.Row

Database: C:\Users\schulz0022\Documents\mining-net\data\mining_segmentation.db
MMAP path: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap


In [40]:
def compute_cluster_id(country_code: str, mining_footprint_json: dict) -> int:
    """Generate a globally unique cluster ID by hashing cluster properties.
    
    SQLite-safe: Returns values in range [0, 2^62) to ensure compatibility with 
    SQLite's signed 64-bit INTEGER type (max value 2^63-1).
    
    Args:
        country_code: ISO3 country code
        mining_footprint_json: GeoJSON of mining footprint geometry
    
    Returns:
        Globally unique cluster ID as positive integer within SQLite INTEGER range
    """
    # Create hash input from country code and mining footprint geometry
    hash_input = json.dumps({
        'country': country_code,
        'footprint': mining_footprint_json
    }, sort_keys=True)
    
    # Generate SHA256 hash and convert first 8 bytes to integer
    hash_obj = hashlib.sha256(hash_input.encode())
    cluster_id = int(hash_obj.hexdigest()[:16], 16)  # Use first 64 bits
    
    # Keep within SQLite's signed INTEGER range with safety margin
    # Use modulo 2^62 to ensure we stay well below 2^63-1 (max SQLite INTEGER)
    # This gives us ~4.6 quintillion unique values, more than enough
    cluster_id = cluster_id % (2**62)
    
    return cluster_id

In [15]:
# Find all directories in mmap that are 3-digit cluster IDs
old_cluster_dirs = []

for item in mmap_path.iterdir():
    if item.is_dir():
        dir_name = item.name
        # Check if it's a 3-digit number (137-146 based on the structure shown)
        if dir_name.isdigit() and len(dir_name) == 3:
            old_cluster_dirs.append((int(dir_name), item))

old_cluster_dirs.sort()
print(f"Found {len(old_cluster_dirs)} old 3-digit cluster ID directories:")
for old_id, path in old_cluster_dirs:
    print(f"  {old_id}: {path.name}")

Found 10 old 3-digit cluster ID directories:
  137: 137
  138: 138
  139: 139
  140: 140
  141: 141
  142: 142
  143: 143
  144: 144
  145: 145
  146: 146


In [16]:
# Query database for tasks with these old cluster IDs
migration_map = {}

for old_cluster_id, old_path in old_cluster_dirs:
    # Get a task with this cluster ID to extract country and mining footprint
    cursor = conn.cursor()
    cursor.execute("""
        SELECT country_code, mining_footprint_json
        FROM tasks
        WHERE cluster_id = ?
        LIMIT 1
    """, (old_cluster_id,))
    
    row = cursor.fetchone()
    
    if row:
        country_code = row['country_code']
        mining_footprint_json = json.loads(row['mining_footprint_json'])
        
        # Compute new cluster ID
        new_cluster_id = compute_cluster_id(country_code, mining_footprint_json)
        
        migration_map[old_cluster_id] = {
            'old_id': old_cluster_id,
            'new_id': new_cluster_id,
            'country': country_code,
            'old_path': old_path,
            'new_path': mmap_path / str(new_cluster_id)
        }
        
        print(f"Cluster {old_cluster_id} ({country_code}) -> {new_cluster_id}")
    else:
        print(f"WARNING: No database entry found for cluster {old_cluster_id}")

print(f"\nTotal migrations planned: {len(migration_map)}")

Cluster 137 (ZAF) -> 7113295390699442408
Cluster 138 (ZAF) -> 4461846190993883110
Cluster 139 (ZAF) -> 1130332382703423897
Cluster 140 (ZAF) -> 8826881135058789811
Cluster 141 (ZAF) -> 5706141593272804103
Cluster 142 (ZAF) -> 7395817511717078632
Cluster 143 (ZAF) -> 6068695432308971879
Cluster 144 (ZAF) -> 4467940815919215314
Cluster 145 (ZAF) -> 5453507745152749462
Cluster 146 (ZAF) -> 8370165278532948664

Total migrations planned: 10


In [17]:
# Check for conflicts (if new directory already exists)
conflicts = []
for old_id, info in migration_map.items():
    if info['new_path'].exists():
        conflicts.append(info)
        print(f"CONFLICT: {info['new_path']} already exists for old cluster {old_id}")

if conflicts:
    print(f"\n⚠️  Found {len(conflicts)} conflicts! Review before proceeding.")
else:
    print("✓ No conflicts found. Safe to proceed with migration.")

✓ No conflicts found. Safe to proceed with migration.


## Perform Migration

Run this cell to rename the directories and update the database.

In [ ]:
# Perform the migration
successful_migrations = []
failed_migrations = []

for old_id, info in migration_map.items():
    try:
        old_path = info['old_path']
        new_path = info['new_path']
        
        # Check if source exists
        if not old_path.exists():
            print(f"ERROR: Source path {old_path} doesn't exist")
            failed_migrations.append((old_id, "Source not found"))
            continue
        
        # Check if destination already exists
        if new_path.exists():
            print(f"SKIP: Destination {new_path} already exists")
            failed_migrations.append((old_id, "Destination exists"))
            continue
        
        # Rename the directory
        print(f"Renaming: {old_path.name} -> {new_path.name}")
        shutil.move(str(old_path), str(new_path))
        
        # Update database using direct connection
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE tasks
            SET cluster_id = ?
            WHERE cluster_id = ?
        """, (info['new_id'], old_id))
        
        # Check if tiles table has the cluster_id column
        cursor.execute("PRAGMA table_info(tiles)")
        tile_columns = {row[1] for row in cursor.fetchall()}
        
        if 'cluster_id' in tile_columns:
            cursor.execute("""
                UPDATE tiles
                SET cluster_id = ?
                WHERE cluster_id = ?
            """, (info['new_id'], old_id))
        
        conn.commit()
        
        successful_migrations.append(old_id)
        print(f"  ✓ Successfully migrated cluster {old_id} -> {info['new_id']}")
        
    except Exception as e:
        print(f"  ✗ Failed to migrate cluster {old_id}: {e}")
        failed_migrations.append((old_id, str(e)))

print(f"\n{'='*60}")
print(f"Migration Summary:")
print(f"  Successful: {len(successful_migrations)}")
print(f"  Failed: {len(failed_migrations)}")
print(f"{'='*60}")

if failed_migrations:
    print("\nFailed migrations:")
    for old_id, error in failed_migrations:
        print(f"  Cluster {old_id}: {error}")

Renaming: 137 -> 7113295390699442408
  ✓ Successfully migrated cluster 137 -> 7113295390699442408
Renaming: 138 -> 4461846190993883110
  ✓ Successfully migrated cluster 138 -> 4461846190993883110
Renaming: 139 -> 1130332382703423897
  ✓ Successfully migrated cluster 139 -> 1130332382703423897
Renaming: 140 -> 8826881135058789811
  ✓ Successfully migrated cluster 140 -> 8826881135058789811
Renaming: 141 -> 5706141593272804103
  ✓ Successfully migrated cluster 141 -> 5706141593272804103
Renaming: 142 -> 7395817511717078632
  ✓ Successfully migrated cluster 142 -> 7395817511717078632
Renaming: 143 -> 6068695432308971879
  ✓ Successfully migrated cluster 143 -> 6068695432308971879
Renaming: 144 -> 4467940815919215314
  ✓ Successfully migrated cluster 144 -> 4467940815919215314
Renaming: 145 -> 5453507745152749462
  ✓ Successfully migrated cluster 145 -> 5453507745152749462
Renaming: 146 -> 8370165278532948664
  ✓ Successfully migrated cluster 146 -> 8370165278532948664

Migration Summary:


## Verify Migration

Check that the old directories are gone and new ones exist.

In [19]:
# Verify old directories are gone
remaining_old = []
for old_id, info in migration_map.items():
    if info['old_path'].exists():
        remaining_old.append(old_id)

if remaining_old:
    print(f"⚠️  {len(remaining_old)} old directories still exist:")
    for old_id in remaining_old:
        print(f"  {old_id}")
else:
    print("✓ All old 3-digit cluster directories have been removed")

# Verify new directories exist
missing_new = []
for old_id, info in migration_map.items():
    if not info['new_path'].exists():
        missing_new.append(old_id)

if missing_new:
    print(f"\n⚠️  {len(missing_new)} new directories are missing:")
    for old_id in missing_new:
        print(f"  {old_id} -> {migration_map[old_id]['new_id']}")
else:
    print("✓ All new cluster directories exist")

# Check database
cursor = conn.cursor()
cursor.execute("SELECT COUNT(DISTINCT cluster_id) FROM tasks WHERE cluster_id < 1000")
old_count = cursor.fetchone()[0]

if old_count > 0:
    print(f"\n⚠️  Database still has {old_count} tasks with old 3-digit cluster IDs")
else:
    print("\n✓ Database has no remaining 3-digit cluster IDs")

# Close connection
conn.close()

✓ All old 3-digit cluster directories have been removed
✓ All new cluster directories exist

⚠️  Database still has 218 tasks with old 3-digit cluster IDs


## Update Database Schema to New Layout

Migrate the legacy database to support new MMAP columns and verify schema.

In [20]:
# Reconnect to database for schema migration
import sqlite3
conn = sqlite3.connect(str(config.DB_PATH))
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

# First, check the current schema of the tiles table
print("Current tiles table schema:")
cursor.execute("PRAGMA table_info(tiles)")
columns = cursor.fetchall()
tile_columns = {row[1] for row in columns}

for col in columns:
    print(f"  {col[1]}: {col[2]}")

print(f"\nColumns present: {tile_columns}")

Current tiles table schema:
  tile_ix: INTEGER
  tile_iy: INTEGER
  geometry_hash: TEXT
  year: INTEGER
  cluster_id: INTEGER
  zarr_written: BOOLEAN
  created_at: TEXT
  written_at: TEXT

Columns present: {'created_at', 'geometry_hash', 'written_at', 'tile_iy', 'tile_ix', 'year', 'zarr_written', 'cluster_id'}


In [21]:
# Migrate schema: Rename zarr columns to mmap and add missing columns if they don't exist
migrations_applied = []

# Handle zarr -> mmap column migration
if 'zarr_written' in tile_columns:
    try:
        print("Renaming zarr_written to mmap_written...")
        # SQLite doesn't support RENAME COLUMN directly in older versions, so we use workaround
        cursor.execute("""
            ALTER TABLE tiles RENAME COLUMN zarr_written TO mmap_written
        """)
        conn.commit()
        migrations_applied.append("zarr_written -> mmap_written")
        print("  ✓ Renamed zarr_written to mmap_written")
        tile_columns.add('mmap_written')
        tile_columns.discard('zarr_written')
    except sqlite3.OperationalError as e:
        print(f"  ⚠️  Error renaming zarr_written: {e}")
        print("  Trying alternative approach...")

if 'written_at' in tile_columns and 'mmap_written_at' not in tile_columns:
    try:
        print("Renaming written_at to mmap_written_at...")
        cursor.execute("""
            ALTER TABLE tiles RENAME COLUMN written_at TO mmap_written_at
        """)
        conn.commit()
        migrations_applied.append("written_at -> mmap_written_at")
        print("  ✓ Renamed written_at to mmap_written_at")
        tile_columns.add('mmap_written_at')
        tile_columns.discard('written_at')
    except sqlite3.OperationalError as e:
        print(f"  ⚠️  Error renaming written_at: {e}")

# Add mmap columns if they don't exist (fallback for older SQLite)
if 'mmap_written' not in tile_columns:
    try:
        print("Adding mmap_written column...")
        cursor.execute("ALTER TABLE tiles ADD COLUMN mmap_written BOOLEAN DEFAULT 0")
        conn.commit()
        migrations_applied.append("mmap_written (added)")
        print("  ✓ Added mmap_written column")
    except sqlite3.OperationalError as e:
        print(f"  ✗ Error adding mmap_written: {e}")

if 'mmap_written_at' not in tile_columns:
    try:
        print("Adding mmap_written_at column...")
        cursor.execute("ALTER TABLE tiles ADD COLUMN mmap_written_at TEXT")
        conn.commit()
        migrations_applied.append("mmap_written_at (added)")
        print("  ✓ Added mmap_written_at column")
    except sqlite3.OperationalError as e:
        print(f"  ✗ Error adding mmap_written_at: {e}")

# Check tasks table for mining_footprint_json
cursor.execute("PRAGMA table_info(tasks)")
task_columns = {row[1] for row in cursor.fetchall()}

if 'mining_footprint_json' not in task_columns:
    try:
        print("Adding mining_footprint_json column to tasks...")
        cursor.execute("ALTER TABLE tasks ADD COLUMN mining_footprint_json TEXT")
        conn.commit()
        migrations_applied.append("tasks.mining_footprint_json")
        print("  ✓ Added mining_footprint_json column")
    except sqlite3.OperationalError as e:
        print(f"  ✗ Error adding mining_footprint_json: {e}")

if 'cluster_id' not in task_columns:
    try:
        print("Adding cluster_id column to tasks...")
        cursor.execute("ALTER TABLE tasks ADD COLUMN cluster_id INTEGER")
        conn.commit()
        migrations_applied.append("tasks.cluster_id")
        print("  ✓ Added cluster_id column")
    except sqlite3.OperationalError as e:
        print(f"  ✗ Error adding cluster_id: {e}")

if migrations_applied:
    print(f"\n✓ Applied {len(migrations_applied)} schema migrations:")
    for m in migrations_applied:
        print(f"  - {m}")
else:
    print("\n✓ Database schema is already up-to-date")

Renaming zarr_written to mmap_written...
  ✓ Renamed zarr_written to mmap_written
Renaming written_at to mmap_written_at...
  ✓ Renamed written_at to mmap_written_at

✓ Applied 2 schema migrations:
  - zarr_written -> mmap_written
  - written_at -> mmap_written_at


In [22]:
# Verify final schema after migrations
print("Final tiles table schema:")
cursor.execute("PRAGMA table_info(tiles)")
columns = cursor.fetchall()

for col in columns:
    print(f"  {col[1]}: {col[2]}")

print("\nFinal tasks table schema:")
cursor.execute("PRAGMA table_info(tasks)")
columns = cursor.fetchall()

# Just show a few key columns
key_cols = ['geometry_hash', 'country_code', 'year', 'cluster_id', 'mining_footprint_json']
for col in columns:
    if col[1] in key_cols:
        print(f"  {col[1]}: {col[2]}")

# Verify no old zarr columns remain
cursor.execute("PRAGMA table_info(tiles)")
final_tile_cols = {row[1] for row in cursor.fetchall()}

if 'zarr_written' in final_tile_cols or 'written_at' in final_tile_cols:
    print("\n⚠️  Old zarr columns still present, manual cleanup may be needed")
else:
    print("\n✓ Old zarr columns successfully migrated to mmap schema")

Final tiles table schema:
  tile_ix: INTEGER
  tile_iy: INTEGER
  geometry_hash: TEXT
  year: INTEGER
  cluster_id: INTEGER
  mmap_written: BOOLEAN
  created_at: TEXT
  mmap_written_at: TEXT

Final tasks table schema:
  geometry_hash: TEXT
  country_code: TEXT
  year: INTEGER
  cluster_id: INTEGER
  mining_footprint_json: TEXT

✓ Old zarr columns successfully migrated to mmap schema


## Update All Cluster IDs to New Hash-Based Format

Update all remaining tasks and tiles with old cluster IDs to the new hash-based format.

In [23]:
# Get all unique (country_code, mining_footprint_json) combinations from tasks
print("Building cluster ID mapping for all tasks...")

cursor.execute("""
    SELECT DISTINCT country_code, mining_footprint_json, cluster_id
    FROM tasks
    WHERE mining_footprint_json IS NOT NULL
    ORDER BY country_code
""")

all_tasks = cursor.fetchall()
cluster_update_map = {}

for task in all_tasks:
    country = task['country_code']
    old_cluster = task['cluster_id']
    
    if task['mining_footprint_json']:
        footprint = json.loads(task['mining_footprint_json'])
        new_cluster = compute_cluster_id(country, footprint)
        
        if old_cluster != new_cluster:
            if old_cluster not in cluster_update_map:
                cluster_update_map[old_cluster] = new_cluster
                print(f"  Old cluster {old_cluster} -> New cluster {new_cluster} ({country})")

print(f"\nTotal cluster ID updates needed: {len(cluster_update_map)}")

Building cluster ID mapping for all tasks...
  Old cluster 0 -> New cluster 4142974597563219461 (ZAF)
  Old cluster 1 -> New cluster 7421833940458184090 (ZAF)
  Old cluster 2 -> New cluster 5472899743927694668 (ZAF)
  Old cluster 3 -> New cluster 620024618185229339 (ZAF)
  Old cluster 4 -> New cluster 4664338972387689494 (ZAF)
  Old cluster 5 -> New cluster 5102590831840184653 (ZAF)
  Old cluster 6 -> New cluster 7700148157382274089 (ZAF)
  Old cluster 7 -> New cluster 3178341026490732864 (ZAF)
  Old cluster 8 -> New cluster 474582821099989246 (ZAF)
  Old cluster 9 -> New cluster 2495649146061856042 (ZAF)
  Old cluster 10 -> New cluster 4856306306860705678 (ZAF)
  Old cluster 11 -> New cluster 71137107360758801 (ZAF)
  Old cluster 12 -> New cluster 5403878321217673907 (ZAF)
  Old cluster 13 -> New cluster 4244490756375463389 (ZAF)
  Old cluster 14 -> New cluster 1609785863007455954 (ZAF)
  Old cluster 15 -> New cluster 5732381289802048234 (ZAF)
  Old cluster 16 -> New cluster 534346556

In [24]:
# Apply cluster ID updates to tasks and tiles tables
updates_applied = {'tasks': 0, 'tiles': 0}

for old_cid, new_cid in cluster_update_map.items():
    # Update tasks table
    cursor.execute("""
        UPDATE tasks
        SET cluster_id = ?
        WHERE cluster_id = ?
    """, (new_cid, old_cid))
    count = cursor.rowcount
    if count > 0:
        updates_applied['tasks'] += count
        print(f"Updated {count} tasks: cluster {old_cid} -> {new_cid}")
    
    # Update tiles table if it has cluster_id column
    cursor.execute("PRAGMA table_info(tiles)")
    tile_cols = {row[1] for row in cursor.fetchall()}
    
    if 'cluster_id' in tile_cols:
        cursor.execute("""
            UPDATE tiles
            SET cluster_id = ?
            WHERE cluster_id = ?
        """, (new_cid, old_cid))
        count = cursor.rowcount
        if count > 0:
            updates_applied['tiles'] += count
            print(f"Updated {count} tiles: cluster {old_cid} -> {new_cid}")

conn.commit()

print(f"\n{'='*60}")
print("Update Summary:")
print(f"  Tasks updated: {updates_applied['tasks']}")
print(f"  Tiles updated: {updates_applied['tiles']}")
print(f"{'='*60}")

Updated 1 tasks: cluster 0 -> 4142974597563219461
Updated 56 tiles: cluster 0 -> 4142974597563219461
Updated 1 tasks: cluster 1 -> 7421833940458184090
Updated 144 tiles: cluster 1 -> 7421833940458184090
Updated 1 tasks: cluster 2 -> 5472899743927694668
Updated 224 tiles: cluster 2 -> 5472899743927694668
Updated 1 tasks: cluster 3 -> 620024618185229339
Updated 273 tiles: cluster 3 -> 620024618185229339
Updated 1 tasks: cluster 4 -> 4664338972387689494
Updated 100 tiles: cluster 4 -> 4664338972387689494
Updated 1 tasks: cluster 5 -> 5102590831840184653
Updated 81 tiles: cluster 5 -> 5102590831840184653
Updated 1 tasks: cluster 6 -> 7700148157382274089
Updated 110 tiles: cluster 6 -> 7700148157382274089
Updated 1 tasks: cluster 7 -> 3178341026490732864
Updated 154 tiles: cluster 7 -> 3178341026490732864
Updated 1 tasks: cluster 8 -> 474582821099989246
Updated 110 tiles: cluster 8 -> 474582821099989246
Updated 1 tasks: cluster 9 -> 2495649146061856042
Updated 90 tiles: cluster 9 -> 2495649

## Verify Database Integrity

Check that all updates were applied correctly and data is consistent.

In [25]:
# Check for any remaining 3-digit cluster IDs
cursor.execute("""
    SELECT COUNT(DISTINCT cluster_id) as count, 'tasks' as table_name
    FROM tasks
    WHERE cluster_id IS NOT NULL AND cluster_id < 1000
    UNION ALL
    SELECT COUNT(DISTINCT cluster_id), 'tiles'
    FROM tiles
    WHERE cluster_id IS NOT NULL AND cluster_id < 1000
""")

results = cursor.fetchall()
remaining_old = {}

for row in results:
    remaining_old[row['table_name']] = row['count']

print("Remaining 3-digit cluster IDs:")
for table, count in remaining_old.items():
    if count > 0:
        print(f"  ⚠️  {table}: {count}")
        # Show which ones
        if table == 'tasks':
            cursor.execute("SELECT DISTINCT cluster_id FROM tasks WHERE cluster_id < 1000 ORDER BY cluster_id")
        else:
            cursor.execute("SELECT DISTINCT cluster_id FROM tiles WHERE cluster_id < 1000 ORDER BY cluster_id")
        ids = [str(row[0]) for row in cursor.fetchall()]
        print(f"     IDs: {', '.join(ids)}")
    else:
        print(f"  ✓ {table}: None")

# Get statistics on new cluster IDs
cursor.execute("SELECT COUNT(DISTINCT cluster_id) FROM tasks WHERE cluster_id IS NOT NULL")
new_cluster_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM tasks WHERE cluster_id IS NOT NULL")
tasks_with_cluster = cursor.fetchone()[0]

print(f"\nNew cluster statistics:")
print(f"  Distinct clusters: {new_cluster_count}")
print(f"  Tasks with cluster ID: {tasks_with_cluster}")

Remaining 3-digit cluster IDs:
  ✓ tasks: None
  ✓ tiles: None

New cluster statistics:
  Distinct clusters: 228
  Tasks with cluster ID: 228


In [26]:
# Verify data consistency between tasks and tiles
print("Verifying data consistency...")

# Check if all tiles reference valid tasks
cursor.execute("""
    SELECT COUNT(*) as orphan_count
    FROM tiles t
    WHERE NOT EXISTS (
        SELECT 1 FROM tasks
        WHERE geometry_hash = t.geometry_hash
        AND year = t.year
    )
""")

orphan_count = cursor.fetchone()['orphan_count']

if orphan_count > 0:
    print(f"  ⚠️  Found {orphan_count} orphaned tile records (no matching task)")
else:
    print(f"  ✓ All tiles have matching tasks")

# Check total rows
cursor.execute("SELECT COUNT(*) as task_count FROM tasks")
task_count = cursor.fetchone()['task_count']

cursor.execute("SELECT COUNT(*) as tile_count FROM tiles")
tile_count = cursor.fetchone()['tile_count']

print(f"\nDatabase summary:")
print(f"  Total tasks: {task_count}")
print(f"  Total tiles: {tile_count}")

# Final status
cursor.execute("SELECT status, COUNT(*) as count FROM tasks GROUP BY status")
status_counts = cursor.fetchall()

print(f"\nTask status distribution:")
for row in status_counts:
    print(f"  {row['status']}: {row['count']}")

# Close connection
conn.close()
print("\n✓ Database migration and verification complete")

Verifying data consistency...
  ✓ All tiles have matching tasks

Database summary:
  Total tasks: 228
  Total tiles: 41797

Task status distribution:
  completed: 2
  pending: 81
  reprojected: 137
  submitted: 8

✓ Database migration and verification complete


## Final Verification: MMAP Tasks and Stale Downloads

Verify that all tasks in MMAP are correctly marked as reprojected and identify stale files in downloads folder.

In [28]:
# Reconnect for final verification
conn = sqlite3.connect(str(config.DB_PATH))
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

print("="*60)
print("Checking MMAP Task Status Consistency")
print("="*60)

# Get all tasks with mmap_written = 1 (tasks that have been reprojected to mmap)
cursor.execute("""
    SELECT COUNT(*) as count
    FROM (
        SELECT DISTINCT t.geometry_hash, t.year
        FROM tiles t
        WHERE t.mmap_written = 1
    )
""")

mmap_tile_count = cursor.fetchone()['count']
print(f"\nTiles written to MMAP: {mmap_tile_count}")

# Get all tasks corresponding to these tiles
cursor.execute("""
    SELECT DISTINCT t.geometry_hash, t.year, ta.status, ta.country_code
    FROM tiles t
    JOIN tasks ta ON t.geometry_hash = ta.geometry_hash AND t.year = ta.year
    WHERE t.mmap_written = 1
    ORDER BY ta.country_code, ta.year
""")

mmap_tasks = cursor.fetchall()
status_check = {'reprojected': 0, 'other': 0}
status_issues = []

for task in mmap_tasks:
    if task['status'] == 'reprojected':
        status_check['reprojected'] += 1
    else:
        status_check['other'] += 1
        status_issues.append({
            'hash': task['geometry_hash'][:8],
            'year': task['year'],
            'country': task['country_code'],
            'status': task['status']
        })

print(f"\nStatus of MMAP tasks:")
print(f"  ✓ Correctly marked 'reprojected': {status_check['reprojected']}")
print(f"  ⚠️  Other status: {status_check['other']}")

if status_issues:
    print(f"\n⚠️  Found {len(status_issues)} tasks with incorrect status:")
    for issue in status_issues[:10]:  # Show first 10
        print(f"    {issue['country']} {issue['year']} ({issue['hash']}...): {issue['status']}")
    if len(status_issues) > 10:
        print(f"    ... and {len(status_issues) - 10} more")
else:
    print("\n✓ All MMAP tasks are correctly marked as 'reprojected'")

Checking MMAP Task Status Consistency

Tiles written to MMAP: 137

Status of MMAP tasks:
  ✓ Correctly marked 'reprojected': 137
  ⚠️  Other status: 0

✓ All MMAP tasks are correctly marked as 'reprojected'


In [29]:
# Check for stale .tif files in downloads folder
print("\n" + "="*60)
print("Checking for Stale Downloaded Files")
print("="*60)

downloads_dir = config.DOWNLOAD_DIR
print(f"\nSearching for .tif files in: {downloads_dir}")

if not downloads_dir.exists():
    print(f"⚠️  Downloads directory does not exist: {downloads_dir}")
else:
    # Find all .tif files
    tif_files = list(downloads_dir.glob("*.tif"))
    print(f"\nFound {len(tif_files)} .tif files in downloads")
    
    if len(tif_files) > 0:
        # Extract task descriptions and check status
        stale_files = []
        
        for tif_file in tif_files:
            filename = tif_file.name
            print(f"\n  File: {filename}")
            print(f"    Size: {tif_file.stat().st_size / (1024**2):.1f} MB")
            
            # Try to match filename to task
            # Format: LANDSAT_C02_T1_L2_{country_code}_{geometry_hash[:8]}_{year}
            parts = filename.replace('.tif', '').split('_')
            
            if len(parts) >= 5:
                try:
                    country_code = parts[4]
                    year = int(parts[-1])
                    
                    # Find matching task
                    cursor.execute("""
                        SELECT geometry_hash, status, downloaded_at
                        FROM tasks
                        WHERE country_code = ? AND year = ?
                        LIMIT 5
                    """, (country_code, year))
                    
                    matching_tasks = cursor.fetchall()
                    
                    if matching_tasks:
                        for task in matching_tasks:
                            status = task['status']
                            hash_short = task['geometry_hash'][:8]
                            
                            if status == 'downloaded':
                                # This file should have been reprojected by now
                                stale_files.append({
                                    'filename': filename,
                                    'country': country_code,
                                    'year': year,
                                    'hash': hash_short,
                                    'status': status,
                                    'downloaded_at': task['downloaded_at']
                                })
                                print(f"    ⚠️  Status: {status} (should be reprojected)")
                            else:
                                print(f"    Status: {status}")
                    else:
                        print(f"    ⚠️  No matching tasks found for {country_code} {year}")
                        stale_files.append({
                            'filename': filename,
                            'country': country_code,
                            'year': year,
                            'hash': '?',
                            'status': 'unknown',
                            'downloaded_at': None
                        })
                except (ValueError, IndexError) as e:
                    print(f"    ⚠️  Could not parse filename: {e}")
        
        # Summary
        print(f"\n{'='*60}")
        print(f"Stale/Stuck Download Summary:")
        
        if stale_files:
            print(f"Found {len(stale_files)} files that may need attention:")
            for f in stale_files:
                print(f"  {f['filename']}")
                if f['downloaded_at']:
                    print(f"    Downloaded: {f['downloaded_at']}")
        else:
            print(f"✓ All downloaded files have been reprojected or are unknown")
    else:
        print("✓ No .tif files found in downloads (clean)")


Checking for Stale Downloaded Files

Searching for .tif files in: C:\Users\schulz0022\Documents\mining-net\data\downloads

Found 16 .tif files in downloads

  File: LANDSAT_C02_T1_L2_ZAF_0516ae77_2019.tif
    Size: 31.7 MB
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected

  File: LANDSAT_C02_T1_L2_ZAF_173793b9_2019.tif
    Size: 98.2 MB
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected

  File: LANDSAT_C02_T1_L2_ZAF_23fcc476_2019.tif
    Size: 13.0 MB
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected

  File: LANDSAT_C02_T1_L2_ZAF_321c9b49_2019.tif
    Size: 19.1 MB
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected
    Status: reprojected

  File: LANDSAT_C02_T1_L2_ZAF_60944937_2019.tif
    Size: 25.5 MB
    Status: reprojected
    

In [30]:
# Generate final migration report
print("\n" + "="*60)
print("MIGRATION COMPLETION REPORT")
print("="*60)

# Summary statistics
cursor.execute("SELECT COUNT(*) as count FROM tasks")
total_tasks = cursor.fetchone()['count']

cursor.execute("SELECT COUNT(*) as count FROM tiles")
total_tiles = cursor.fetchone()['count']

cursor.execute("SELECT COUNT(*) as count FROM tasks WHERE status = 'reprojected'")
reprojected_tasks = cursor.fetchone()['count']

cursor.execute("SELECT COUNT(DISTINCT cluster_id) as count FROM tasks WHERE cluster_id IS NOT NULL")
total_clusters = cursor.fetchone()['count']

cursor.execute("""
    SELECT COUNT(DISTINCT cluster_id) as count FROM tasks 
    WHERE cluster_id < 1000
""")
remaining_old_clusters = cursor.fetchone()['count']

print(f"\nDatabase Statistics:")
print(f"  Total tasks: {total_tasks}")
print(f"  Total tiles: {total_tiles}")
print(f"  Tasks marked reprojected: {reprojected_tasks}")
print(f"  Total clusters (new format): {total_clusters}")
print(f"  Remaining 3-digit clusters: {remaining_old_clusters}")

# Check task status distribution
cursor.execute("""
    SELECT status, COUNT(*) as count 
    FROM tasks 
    GROUP BY status 
    ORDER BY count DESC
""")

print(f"\nTask Status Distribution:")
for row in cursor.fetchall():
    print(f"  {row['status']}: {row['count']}")

# Migration success indicators
print(f"\n{'='*60}")
print("Migration Success Checklist:")
print(f"{'='*60}")

checks = []

# Check 1: Old directories renamed
checks.append(('Old 3-digit directories removed', remaining_old_clusters == 0))

# Check 2: Schema migrated
cursor.execute("PRAGMA table_info(tiles)")
tile_cols = {row[1] for row in cursor.fetchall()}
has_mmap_cols = 'mmap_written' in tile_cols
checks.append(('MMAP schema columns present', has_mmap_cols))

# Check 3: Cluster IDs updated
cursor.execute("SELECT COUNT(*) as count FROM tasks WHERE cluster_id > 1000")
new_cluster_count = cursor.fetchone()['count']
checks.append(('Tasks use new cluster IDs', new_cluster_count > 0))

# Check 4: MMAP tiles indexed
cursor.execute("SELECT COUNT(*) as count FROM tiles WHERE mmap_written = 1")
mmap_indexed = cursor.fetchone()['count']
checks.append(('MMAP tiles indexed', mmap_indexed > 0))

# Check 5: No orphaned tiles
cursor.execute("""
    SELECT COUNT(*) as count FROM tiles t
    WHERE NOT EXISTS (
        SELECT 1 FROM tasks WHERE geometry_hash = t.geometry_hash AND year = t.year
    )
""")
orphaned = cursor.fetchone()['count']
checks.append(('No orphaned tiles', orphaned == 0))

for check_name, passed in checks:
    status = '✓' if passed else '✗'
    print(f"{status} {check_name}")

all_passed = all(p for _, p in checks)
print(f"\n{'='*60}")
if all_passed:
    print("✓ MIGRATION COMPLETE AND SUCCESSFUL")
else:
    print("⚠️  MIGRATION COMPLETE WITH ISSUES - REVIEW ABOVE")
print(f"{'='*60}")

conn.close()


MIGRATION COMPLETION REPORT

Database Statistics:
  Total tasks: 228
  Total tiles: 41797
  Tasks marked reprojected: 137
  Total clusters (new format): 228
  Remaining 3-digit clusters: 0

Task Status Distribution:
  reprojected: 137
  pending: 81
  submitted: 8
  completed: 2

Migration Success Checklist:
✓ Old 3-digit directories removed
✓ MMAP schema columns present
✓ Tasks use new cluster IDs
✓ MMAP tiles indexed
✓ No orphaned tiles

✓ MIGRATION COMPLETE AND SUCCESSFUL


## Reconcile MMAP Disk vs Database

Verify that all cluster directories on disk are found in the database. If missing, resolve by inspecting tile subfolders and matching to database entries. Dry-run first.

In [35]:
# Reconnect to database
conn = sqlite3.connect(str(config.DB_PATH))
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

print("="*60)
print("Reconciling MMAP Disk vs Database")
print("="*60)

# Get all cluster directories on disk
disk_cluster_dirs = []
if mmap_path.exists():
    for item in mmap_path.iterdir():
        if item.is_dir() and item.name.isdigit():
            try:
                cluster_id = int(item.name)
                disk_cluster_dirs.append((cluster_id, item))
            except ValueError:
                pass

disk_cluster_dirs.sort()
print(f"\nCluster directories found on disk: {len(disk_cluster_dirs)}")
for cid, path in disk_cluster_dirs[:10]:
    print(f"  {cid}: {path.name}")
if len(disk_cluster_dirs) > 10:
    print(f"  ... and {len(disk_cluster_dirs) - 10} more")

# Get all cluster IDs in database
cursor.execute("SELECT DISTINCT cluster_id FROM tiles WHERE cluster_id IS NOT NULL ORDER BY cluster_id")
db_cluster_ids = set(row['cluster_id'] for row in cursor.fetchall())
print(f"\nCluster IDs in database: {len(db_cluster_ids)}")

# Compare
missing_in_db = []
for cid, path in disk_cluster_dirs:
    if cid not in db_cluster_ids:
        missing_in_db.append((cid, path))

if missing_in_db:
    print(f"\n⚠️  Found {len(missing_in_db)} cluster directories NOT in database:")
    for cid, path in missing_in_db[:10]:
        print(f"  {cid}: {path}")
    if len(missing_in_db) > 10:
        print(f"  ... and {len(missing_in_db) - 10} more")
else:
    print(f"\n✓ All disk cluster directories are in database")

print(f"\nDatabase-only clusters (no disk directory):")
disk_cluster_ids = set(cid for cid, _ in disk_cluster_dirs)
db_only = db_cluster_ids - disk_cluster_ids
print(f"  Count: {len(db_only)}")
if db_only:
    for cid in sorted(list(db_only))[:10]:
        print(f"    {cid}")
    if len(db_only) > 10:
        print(f"    ... and {len(db_only) - 10} more")

Reconciling MMAP Disk vs Database

Cluster directories found on disk: 147
  71137107360758801: 71137107360758801
  153188716851427413: 153188716851427413
  322565366526356651: 322565366526356651
  969356297909971018: 969356297909971018
  1003308299838755742: 1003308299838755742
  1028026081820681689: 1028026081820681689
  1130332382703423897: 1130332382703423897
  1531402764234967612: 1531402764234967612
  1602712004546264064: 1602712004546264064
  1608417650764473188: 1608417650764473188
  ... and 137 more

Cluster IDs in database: 228

⚠️  Found 77 cluster directories NOT in database:
  9240086712580376533: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9240086712580376533
  9240687433499654889: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9240687433499654889
  9531543105325859061: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9531543105325859061
  9548364359867733486: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\954836435986773348

In [37]:
# Try to resolve missing cluster IDs by inspecting tile subfolders
if missing_in_db:
    print("\n" + "="*60)
    print("Attempting to Resolve Missing Clusters")
    print("="*60)
    
    resolution_map = {}
    
    for unknown_cid, cluster_path in missing_in_db:
        print(f"\nInvestigating cluster {unknown_cid} at {cluster_path}")
        
        # Look for year subdirectories first, then tile subdirectories
        tile_dirs = []
        year_dirs = []
        
        for item in cluster_path.iterdir():
            if item.is_dir():
                # Check for year directories (4 digits, 19xx or 20xx)
                if item.name.isdigit() and len(item.name) == 4:
                    year_dirs.append(item)
        
        if year_dirs:
            print(f"  Found {len(year_dirs)} year directories: {', '.join(d.name for d in year_dirs[:5])}")
            
            # Look for tile directories in year directories
            for year_dir in year_dirs:
                for item in year_dir.iterdir():
                    if item.is_dir():
                        dir_name = item.name
                        # Try to parse as ix_iy format
                        if '_' in dir_name:
                            parts = dir_name.split('_')
                            if len(parts) == 2:
                                try:
                                    ix, iy = int(parts[0]), int(parts[1])
                                    year = int(year_dir.name)
                                    tile_dirs.append((ix, iy, year, item))
                                except ValueError:
                                    pass
        
        if tile_dirs:
            print(f"  Found {len(tile_dirs)} tile directories with coordinates")
            # Get a sample of coordinates
            for ix, iy, year, _ in tile_dirs[:3]:
                print(f"    Tile {ix}_{iy} (year {year})")
            if len(tile_dirs) > 3:
                print(f"    ... and {len(tile_dirs) - 3} more tiles")
            
            # Search database for tiles with these coordinates
            ix, iy, year, _ = tile_dirs[0]  # Use first tile as reference
            cursor.execute("""
                SELECT DISTINCT t.cluster_id, ta.country_code, t.year
                FROM tiles t
                JOIN tasks ta ON t.geometry_hash = ta.geometry_hash AND t.year = ta.year
                WHERE t.tile_ix = ? AND t.tile_iy = ? AND t.year = ?
                LIMIT 5
            """, (ix, iy, year))
            
            matches = cursor.fetchall()
            
            if matches:
                print(f"  Found matching tiles in database:")
                for match in matches:
                    print(f"    Cluster {match['cluster_id']}: {match['country_code']} {match['year']}")
                    # This is likely the correct cluster ID
                    resolution_map[unknown_cid] = match['cluster_id']
            else:
                print(f"  ⚠️  No matching tiles found for coordinates ({ix}, {iy}, year {year})")
                # Try broader search without year constraint
                cursor.execute("""
                    SELECT DISTINCT t.cluster_id, ta.country_code, t.year
                    FROM tiles t
                    JOIN tasks ta ON t.geometry_hash = ta.geometry_hash AND t.year = ta.year
                    WHERE t.tile_ix = ? AND t.tile_iy = ?
                    LIMIT 5
                """, (ix, iy))
                
                broader_matches = cursor.fetchall()
                if broader_matches:
                    print(f"  Found matches with different years:")
                    for match in broader_matches:
                        print(f"    Cluster {match['cluster_id']}: {match['country_code']} {match['year']}")
                    resolution_map[unknown_cid] = broader_matches[0]['cluster_id']
        else:
            print(f"  ⚠️  No tile directories with coordinates found")
    
    print(f"\n{'='*60}")
    print(f"Resolution Summary:")
    print(f"  Resolved: {len(resolution_map)}")
    print(f"  Unresolved: {len(missing_in_db) - len(resolution_map)}")
    
    if resolution_map:
        print(f"\nProposed Renames (DRY RUN):")
        for old_cid, new_cid in resolution_map.items():
            old_path = mmap_path / str(old_cid)
            new_path = mmap_path / str(new_cid)
            print(f"  {old_cid} -> {new_cid}")
            print(f"    From: {old_path}")
            print(f"    To:   {new_path}")
            print(f"    Conflict: {new_path.exists()}")
else:
    print("\n✓ No missing clusters to resolve")
    resolution_map = {}


Attempting to Resolve Missing Clusters

Investigating cluster 9240086712580376533 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9240086712580376533
  Found 1 year directories: 2019
  Found 169 tile directories with coordinates
    Tile 6598_12208 (year 2019)
    Tile 6598_12209 (year 2019)
    Tile 6598_12210 (year 2019)
    ... and 166 more tiles
  Found matching tiles in database:
    Cluster 16714675725600725: ZAF 2019

Investigating cluster 9240687433499654889 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9240687433499654889
  Found 1 year directories: 2019
  Found 110 tile directories with coordinates
    Tile 6872_11827 (year 2019)
    Tile 6872_11828 (year 2019)
    Tile 6872_11829 (year 2019)
    ... and 107 more tiles
  Found matching tiles in database:
    Cluster 17315396644879081: ZAF 2019

Investigating cluster 9531543105325859061 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\9531543105325859061
  Found 1 year directories: 2

In [39]:
# Execute reconciliation (uncomment to actually rename directories and update database)
print("\n" + "="*60)
print("RECONCILIATION - READY TO EXECUTE")
print("="*60)

if resolution_map:
    print(f"\nTo execute the {len(resolution_map)} renames and database updates:")
    print("1. Review the proposed changes above")
    print("2. Uncomment the code below")
    print("3. Re-run this cell")
    
    # UNCOMMENT THE FOLLOWING TO EXECUTE:

    execute_reconciliation = True
    
    if execute_reconciliation:
        successful = []
        failed = []
        
        for old_cid, new_cid in resolution_map.items():
            try:
                old_path = mmap_path / str(old_cid)
                new_path = mmap_path / str(new_cid)
                
                # Check conflicts
                if new_path.exists():
                    print(f"⚠️  Conflict: {new_path} already exists, skipping")
                    failed.append((old_cid, "Destination exists"))
                    continue
                
                # Rename directory
                print(f"Renaming: {old_cid} -> {new_cid}")
                shutil.move(str(old_path), str(new_path))
                
                # Update database
                cursor.execute('''
                    UPDATE tiles
                    SET cluster_id = ?
                    WHERE cluster_id = ?
                ''', (new_cid, old_cid))
                
                conn.commit()
                successful.append(old_cid)
                print(f"  ✓ Success")
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
                failed.append((old_cid, str(e)))
        
        print(f"\n{'='*60}")
        print(f"Reconciliation Complete:")
        print(f"  Successful: {len(successful)}")
        print(f"  Failed: {len(failed)}")
        print(f"{'='*60}")

    
else:
    print("\n✓ No reconciliation needed - all disk clusters are in database")


RECONCILIATION - READY TO EXECUTE

To execute the 77 renames and database updates:
1. Review the proposed changes above
2. Uncomment the code below
3. Re-run this cell
Renaming: 9240086712580376533 -> 16714675725600725
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9240687433499654889 -> 17315396644879081
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9531543105325859061 -> 308171068471083253
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9548364359867733486 -> 324992323012957678
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9551308160817396001 -> 327936123962620193
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9632786344630052084 -> 409414307775276276
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9647079569036913146 -> 423707532182137338
  ✗ Error: Python int too large to convert to SQLite INTEGER
Renaming: 9678285412521050395 -> 454913375

## Fix SQLite INTEGER Overflow

The previous cluster IDs exceeded SQLite's signed 64-bit INTEGER limit (2^63-1). We need to:
1. Update the hash function to use 2^62 modulo (safe for SQLite)
2. Recompute all cluster IDs in the database
3. Rename all MMAP directories to match new IDs

In [41]:
# Build mapping of old cluster IDs to new SQLite-safe IDs
print("="*60)
print("Building Old -> New Cluster ID Mapping")
print("="*60)

cursor.execute("""
    SELECT DISTINCT country_code, mining_footprint_json, cluster_id
    FROM tasks
    WHERE mining_footprint_json IS NOT NULL
    ORDER BY cluster_id
""")

all_clusters = cursor.fetchall()
old_to_new_map = {}

print(f"\nProcessing {len(all_clusters)} unique clusters...")

for cluster in all_clusters:
    country = cluster['country_code']
    old_cid = cluster['cluster_id']
    
    if cluster['mining_footprint_json']:
        footprint = json.loads(cluster['mining_footprint_json'])
        new_cid = compute_cluster_id(country, footprint)
        
        if old_cid != new_cid:
            old_to_new_map[old_cid] = new_cid
            print(f"  {old_cid} -> {new_cid} ({country})")

print(f"\n{'='*60}")
print(f"Cluster IDs to update: {len(old_to_new_map)}")
print(f"Max new cluster ID: {max(old_to_new_map.values()) if old_to_new_map else 0}")
print(f"SQLite max INTEGER: {2**63 - 1}")
print(f"Safe threshold (2^62): {2**62}")
print(f"{'='*60}")

Building Old -> New Cluster ID Mapping

Processing 228 unique clusters...
  4617841705973987919 -> 6155687546600015 (ZAF)
  4664338972387689494 -> 52652953960301590 (ZAF)
  4689177428417318367 -> 77491409989930463 (ZAF)
  4700708545171296629 -> 89022526743908725 (ZAF)
  4713220901711428553 -> 101534883284040649 (ZAF)
  4741925331851115913 -> 130239313423728009 (ZAF)
  4856306306860705678 -> 244620288433317774 (ZAF)
  4887119614678085029 -> 275433596250697125 (ZAF)
  4909717510893952789 -> 298031492466564885 (ZAF)
  4995195210052363224 -> 383509191624975320 (ZAF)
  5034849909358132987 -> 423163890930745083 (ZAF)
  5102590831840184653 -> 490904813412796749 (ZAF)
  5104318938202514235 -> 492632919775126331 (ZAF)
  5108604503963515421 -> 496918485536127517 (ZAF)
  5146548221802567392 -> 534862203375179488 (ZAF)
  5202365793440106789 -> 590679775012718885 (ZAF)
  5218828158162335319 -> 607142139734947415 (ZAF)
  5312754078679895004 -> 701068060252507100 (ZAF)
  5326181876138131612 -> 714495

In [42]:
# Update database with new SQLite-safe cluster IDs
print("\n" + "="*60)
print("Updating Database with SQLite-Safe Cluster IDs")
print("="*60)

db_updates = {'tasks': 0, 'tiles': 0}

for old_cid, new_cid in old_to_new_map.items():
    # Update tasks table
    cursor.execute("""
        UPDATE tasks
        SET cluster_id = ?
        WHERE cluster_id = ?
    """, (new_cid, old_cid))
    
    tasks_updated = cursor.rowcount
    if tasks_updated > 0:
        db_updates['tasks'] += tasks_updated
        print(f"Tasks: {old_cid} -> {new_cid} ({tasks_updated} rows)")
    
    # Update tiles table
    cursor.execute("""
        UPDATE tiles
        SET cluster_id = ?
        WHERE cluster_id = ?
    """, (new_cid, old_cid))
    
    tiles_updated = cursor.rowcount
    if tiles_updated > 0:
        db_updates['tiles'] += tiles_updated
        print(f"Tiles: {old_cid} -> {new_cid} ({tiles_updated} rows)")

conn.commit()

print(f"\n{'='*60}")
print("Database Update Summary:")
print(f"  Tasks rows updated: {db_updates['tasks']}")
print(f"  Tiles rows updated: {db_updates['tiles']}")
print(f"{'='*60}")


Updating Database with SQLite-Safe Cluster IDs
Tasks: 4617841705973987919 -> 6155687546600015 (1 rows)
Tiles: 4617841705973987919 -> 6155687546600015 (72 rows)
Tasks: 4664338972387689494 -> 52652953960301590 (1 rows)
Tiles: 4664338972387689494 -> 52652953960301590 (100 rows)
Tasks: 4689177428417318367 -> 77491409989930463 (1 rows)
Tiles: 4689177428417318367 -> 77491409989930463 (110 rows)
Tasks: 4700708545171296629 -> 89022526743908725 (1 rows)
Tiles: 4700708545171296629 -> 89022526743908725 (92 rows)
Tasks: 4713220901711428553 -> 101534883284040649 (1 rows)
Tiles: 4713220901711428553 -> 101534883284040649 (121 rows)
Tasks: 4741925331851115913 -> 130239313423728009 (1 rows)
Tiles: 4741925331851115913 -> 130239313423728009 (744 rows)
Tasks: 4856306306860705678 -> 244620288433317774 (1 rows)
Tiles: 4856306306860705678 -> 244620288433317774 (110 rows)
Tasks: 4887119614678085029 -> 275433596250697125 (1 rows)
Tiles: 4887119614678085029 -> 275433596250697125 (120 rows)
Tasks: 4909717510893

In [43]:
# Rename MMAP directories to match new SQLite-safe cluster IDs
print("\n" + "="*60)
print("Renaming MMAP Directories")
print("="*60)

mmap_renames = {'successful': [], 'failed': [], 'skipped': []}

for old_cid, new_cid in old_to_new_map.items():
    old_path = mmap_path / str(old_cid)
    new_path = mmap_path / str(new_cid)
    
    if not old_path.exists():
        # This cluster doesn't have MMAP data, skip
        continue
    
    if new_path.exists():
        print(f"⚠️  Conflict: {new_path.name} already exists")
        mmap_renames['skipped'].append((old_cid, new_cid, "Destination exists"))
        continue
    
    try:
        print(f"Renaming: {old_cid} -> {new_cid}")
        shutil.move(str(old_path), str(new_path))
        mmap_renames['successful'].append((old_cid, new_cid))
        print(f"  ✓ Success")
    except Exception as e:
        print(f"  ✗ Error: {e}")
        mmap_renames['failed'].append((old_cid, new_cid, str(e)))

print(f"\n{'='*60}")
print("MMAP Directory Rename Summary:")
print(f"  Successful: {len(mmap_renames['successful'])}")
print(f"  Skipped (conflicts): {len(mmap_renames['skipped'])}")
print(f"  Failed: {len(mmap_renames['failed'])}")
print(f"{'='*60}")

if mmap_renames['failed']:
    print("\nFailed renames:")
    for old_cid, new_cid, error in mmap_renames['failed']:
        print(f"  {old_cid} -> {new_cid}: {error}")


Renaming MMAP Directories
Renaming: 4617841705973987919 -> 6155687546600015
  ✓ Success
Renaming: 4664338972387689494 -> 52652953960301590
  ✓ Success
Renaming: 4689177428417318367 -> 77491409989930463
  ✓ Success
Renaming: 4713220901711428553 -> 101534883284040649
  ✓ Success
Renaming: 4741925331851115913 -> 130239313423728009
  ✓ Success
Renaming: 4856306306860705678 -> 244620288433317774
  ✓ Success
Renaming: 4887119614678085029 -> 275433596250697125
  ✓ Success
Renaming: 4995195210052363224 -> 383509191624975320
  ✓ Success
Renaming: 5034849909358132987 -> 423163890930745083
  ✓ Success
Renaming: 5102590831840184653 -> 490904813412796749
  ✓ Success
Renaming: 5218828158162335319 -> 607142139734947415
  ✓ Success
Renaming: 5312754078679895004 -> 701068060252507100
  ✓ Success
Renaming: 5343465566555283670 -> 731779548127895766
  ✓ Success
Renaming: 5403878321217673907 -> 792192302790286003
  ✓ Success
Renaming: 5453507745152749462 -> 841821726725361558
  ✓ Success
Renaming: 5472899

In [44]:
# Re-run reconciliation with SQLite-safe IDs
print("\n" + "="*60)
print("Re-Running Disk vs Database Reconciliation")
print("="*60)

# Get all cluster directories on disk
disk_cluster_dirs = []
if mmap_path.exists():
    for item in mmap_path.iterdir():
        if item.is_dir() and item.name.isdigit():
            try:
                cluster_id = int(item.name)
                disk_cluster_dirs.append((cluster_id, item))
            except ValueError:
                pass

disk_cluster_dirs.sort()
print(f"\nCluster directories on disk: {len(disk_cluster_dirs)}")

# Get all cluster IDs in database
cursor.execute("SELECT DISTINCT cluster_id FROM tiles WHERE cluster_id IS NOT NULL ORDER BY cluster_id")
db_cluster_ids = set(row['cluster_id'] for row in cursor.fetchall())
print(f"Cluster IDs in database: {len(db_cluster_ids)}")

# Compare
disk_cluster_ids = set(cid for cid, _ in disk_cluster_dirs)
missing_in_db = disk_cluster_ids - db_cluster_ids
db_only = db_cluster_ids - disk_cluster_ids

print(f"\nMissing in database: {len(missing_in_db)}")
if missing_in_db:
    for cid in sorted(list(missing_in_db))[:10]:
        print(f"  {cid}")
    if len(missing_in_db) > 10:
        print(f"  ... and {len(missing_in_db) - 10} more")

print(f"\nDatabase-only (no disk): {len(db_only)}")
if db_only:
    for cid in sorted(list(db_only))[:10]:
        print(f"  {cid}")
    if len(db_only) > 10:
        print(f"  ... and {len(db_only) - 10} more")

if not missing_in_db:
    print("\n✓ All disk clusters are now in database!")


Re-Running Disk vs Database Reconciliation

Cluster directories on disk: 147
Cluster IDs in database: 228

Missing in database: 5
  10844206362919472562
  16680566798134402095
  17118050217505876374
  17162516979381748491
  17684285651992456581

Database-only (no disk): 86
  62084203559214856
  89022526743908725
  176191065995097590
  298031492466564885
  306125387993476299
  320860650556975719
  492632919775126331
  496918485536127517
  534862203375179488
  539463802049000380
  ... and 76 more


In [45]:
# Resolve remaining clusters on disk that aren't in database
if missing_in_db:
    print("\n" + "="*60)
    print("Resolving Remaining Missing Clusters")
    print("="*60)
    
    resolution_map = {}
    missing_list = [(cid, mmap_path / str(cid)) for cid in missing_in_db]
    
    for unknown_cid, cluster_path in missing_list:
        print(f"\nInvestigating cluster {unknown_cid} at {cluster_path}")
        
        # Look for year subdirectories first, then tile subdirectories
        tile_dirs = []
        year_dirs = []
        
        for item in cluster_path.iterdir():
            if item.is_dir():
                # Check for year directories (4 digits, 19xx or 20xx)
                if item.name.isdigit() and len(item.name) == 4:
                    year_dirs.append(item)
        
        if year_dirs:
            print(f"  Found {len(year_dirs)} year directories: {', '.join(d.name for d in year_dirs[:5])}")
            
            # Look for tile directories in year directories
            for year_dir in year_dirs:
                for item in year_dir.iterdir():
                    if item.is_dir():
                        dir_name = item.name
                        # Try to parse as ix_iy format
                        if '_' in dir_name:
                            parts = dir_name.split('_')
                            if len(parts) == 2:
                                try:
                                    ix, iy = int(parts[0]), int(parts[1])
                                    year = int(year_dir.name)
                                    tile_dirs.append((ix, iy, year, item))
                                except ValueError:
                                    pass
        
        if tile_dirs:
            print(f"  Found {len(tile_dirs)} tile directories with coordinates")
            # Get a sample of coordinates
            for ix, iy, year, _ in tile_dirs[:3]:
                print(f"    Tile {ix}_{iy} (year {year})")
            if len(tile_dirs) > 3:
                print(f"    ... and {len(tile_dirs) - 3} more tiles")
            
            # Search database for tiles with these coordinates
            ix, iy, year, _ = tile_dirs[0]  # Use first tile as reference
            cursor.execute("""
                SELECT DISTINCT t.cluster_id, ta.country_code, t.year
                FROM tiles t
                JOIN tasks ta ON t.geometry_hash = ta.geometry_hash AND t.year = ta.year
                WHERE t.tile_ix = ? AND t.tile_iy = ? AND t.year = ?
                LIMIT 5
            """, (ix, iy, year))
            
            matches = cursor.fetchall()
            
            if matches:
                print(f"  Found matching tiles in database:")
                for match in matches:
                    print(f"    Cluster {match['cluster_id']}: {match['country_code']} {match['year']}")
                    # This is likely the correct cluster ID
                    resolution_map[unknown_cid] = match['cluster_id']
                    break
            else:
                print(f"  ⚠️  No matching tiles found for coordinates ({ix}, {iy}, year {year})")
                # Try broader search without year constraint
                cursor.execute("""
                    SELECT DISTINCT t.cluster_id, ta.country_code, t.year
                    FROM tiles t
                    JOIN tasks ta ON t.geometry_hash = ta.geometry_hash AND t.year = ta.year
                    WHERE t.tile_ix = ? AND t.tile_iy = ?
                    LIMIT 5
                """, (ix, iy))
                
                broader_matches = cursor.fetchall()
                if broader_matches:
                    print(f"  Found matches with different years:")
                    for match in broader_matches:
                        print(f"    Cluster {match['cluster_id']}: {match['country_code']} {match['year']}")
                    resolution_map[unknown_cid] = broader_matches[0]['cluster_id']
                    break
        else:
            print(f"  ⚠️  No tile directories with coordinates found")
    
    print(f"\n{'='*60}")
    print(f"Resolution Summary:")
    print(f"  Resolved: {len(resolution_map)}")
    print(f"  Unresolved: {len(missing_in_db) - len(resolution_map)}")
    
    if resolution_map:
        print(f"\nProposed Renames:")
        for old_cid, new_cid in resolution_map.items():
            old_path = mmap_path / str(old_cid)
            new_path = mmap_path / str(new_cid)
            print(f"  {old_cid} -> {new_cid}")
            print(f"    Conflict: {new_path.exists()}")
else:
    print("\n✓ All disk clusters are in database!")
    resolution_map = {}


Resolving Remaining Missing Clusters

Investigating cluster 17684285651992456581 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\17684285651992456581
  Found 1 year directories: 2019
  Found 1450 tile directories with coordinates
    Tile 6727_12013 (year 2019)
    Tile 6727_12014 (year 2019)
    Tile 6727_12015 (year 2019)
    ... and 1447 more tiles
  Found matching tiles in database:
    Cluster 3849227596710292869: ZAF 2019

Investigating cluster 17162516979381748491 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\17162516979381748491
  Found 1 year directories: 2019
  Found 180 tile directories with coordinates
    Tile 6850_11403 (year 2019)
    Tile 6850_11404 (year 2019)
    Tile 6850_11405 (year 2019)
    ... and 177 more tiles
  Found matching tiles in database:
    Cluster 3327458924099584779: ZAF 2019

Investigating cluster 10844206362919472562 at C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap\10844206362919472562
  Found 1 year dire

In [47]:
# Execute final renames for remaining clusters
if resolution_map:
    print("\n" + "="*60)
    print("Executing Final Cluster Renames")
    print("="*60)
    
    final_renames = {'successful': [], 'failed': [], 'merged': []}
    
    for old_cid, new_cid in resolution_map.items():
        old_path = mmap_path / str(old_cid)
        new_path = mmap_path / str(new_cid)
        
        if not old_path.exists():
            print(f"⚠️  Source missing: {old_cid}")
            final_renames['failed'].append((old_cid, new_cid, "Source missing"))
            continue
        
        if new_path.exists():
            # The cluster already exists - we need to merge tile data
            # For now, just update the database references
            print(f"Merging: {old_cid} -> {new_cid} (destination exists)")
            try:
                cursor.execute("""
                    UPDATE tiles
                    SET cluster_id = ?
                    WHERE cluster_id = ?
                """, (new_cid, old_cid))
                conn.commit()
                final_renames['merged'].append((old_cid, new_cid))
                print(f"  ✓ Database updated, old directory preserved for manual merge")
            except Exception as e:
                print(f"  ✗ Error: {e}")
                final_renames['failed'].append((old_cid, new_cid, str(e)))
        else:
            # Simple rename
            try:
                print(f"Renaming: {old_cid} -> {new_cid}")
                shutil.move(str(old_path), str(new_path))
                
                # Update database
                cursor.execute("""
                    UPDATE tiles
                    SET cluster_id = ?
                    WHERE cluster_id = ?
                """, (new_cid, old_cid))
                conn.commit()
                
                final_renames['successful'].append((old_cid, new_cid))
                print(f"  ✓ Success")
            except Exception as e:
                print(f"  ✗ Error: {e}")
                final_renames['failed'].append((old_cid, new_cid, str(e)))
    
    print(f"\n{'='*60}")
    print("Final Rename Summary:")
    print(f"  Successful: {len(final_renames['successful'])}")
    print(f"  Merged (dest exists): {len(final_renames['merged'])}")
    print(f"  Failed: {len(final_renames['failed'])}")
    print(f"{'='*60}")
    
    if final_renames['merged']:
        print("\nMerged clusters (manual merge may be needed):")
        for old_cid, new_cid in final_renames['merged']:
            print(f"  Old: {mmap_path / str(old_cid)}")
            print(f"  New: {mmap_path / str(new_cid)}")
else:
    print("\n✓ No renames needed")


Executing Final Cluster Renames
⚠️  Source missing: 17684285651992456581
Merging: 17162516979381748491 -> 3327458924099584779 (destination exists)
  ✗ Error: Python int too large to convert to SQLite INTEGER
⚠️  Source missing: 10844206362919472562
⚠️  Source missing: 16680566798134402095
⚠️  Source missing: 17118050217505876374

Final Rename Summary:
  Successful: 0
  Merged (dest exists): 0
  Failed: 5


In [50]:
# Final verification - check what's on disk vs database
print("\n" + "="*60)
print("FINAL VERIFICATION")
print("="*60)

# Get current state
disk_cluster_dirs = []
if mmap_path.exists():
    for item in mmap_path.iterdir():
        if item.is_dir() and item.name.isdigit():
            try:
                cluster_id = int(item.name)
                # Only include if it fits in SQLite range
                if cluster_id < 2**62:
                    disk_cluster_dirs.append(cluster_id)
                else:
                    print(f"⚠️  Oversized cluster on disk: {cluster_id}")
            except ValueError:
                pass

disk_cluster_dirs.sort()

# Get database clusters
cursor.execute("SELECT DISTINCT cluster_id FROM tiles WHERE cluster_id IS NOT NULL ORDER BY cluster_id")
db_cluster_ids = set(row['cluster_id'] for row in cursor.fetchall())

# Verify max values
max_db_cluster = max(db_cluster_ids) if db_cluster_ids else 0
max_disk_cluster = max(disk_cluster_dirs) if disk_cluster_dirs else 0

print(f"\n✓ Clusters on disk: {len(disk_cluster_dirs)}")
print(f"✓ Clusters in database: {len(db_cluster_ids)}")
print(f"✓ Max disk cluster ID: {max_disk_cluster:,}")
print(f"✓ Max database cluster ID: {max_db_cluster:,}")
print(f"✓ SQLite safe threshold (2^62): {2**62:,}")
print(f"✓ All cluster IDs are SQLite-safe: {max_disk_cluster < 2**62 and max_db_cluster < 2**62}")

# Check for mismatches
disk_set = set(disk_cluster_dirs)
missing_in_db = disk_set - db_cluster_ids
db_only = db_cluster_ids - disk_set

if missing_in_db:
    print(f"\n⚠️  {len(missing_in_db)} clusters on disk NOT in database")
    for cid in sorted(list(missing_in_db))[:10]:
        print(f"    {cid}")
else:
    print(f"\n✓ All disk clusters are in database!")

if db_only:
    print(f"\n📊 {len(db_only)} clusters in database without disk directory (expected for some)")
else:
    print(f"\n✓ All database clusters have disk directories")

print(f"\n{'='*60}")
if not missing_in_db and max_disk_cluster < 2**62 and max_db_cluster < 2**62:
    print("✅ MIGRATION COMPLETE AND SUCCESSFUL!")
    print("   - All cluster IDs are SQLite-safe")
    print("   - All disk directories match database")
else:
    print("⚠️  MIGRATION NEEDS ATTENTION - see issues above")
print(f"{'='*60}")


FINAL VERIFICATION

✓ Clusters on disk: 146
✓ Clusters in database: 228
✓ Max disk cluster ID: 4,573,129,916,222,891,577
✓ Max database cluster ID: 4,589,502,480,873,092,105
✓ SQLite safe threshold (2^62): 4,611,686,018,427,387,904
✓ All cluster IDs are SQLite-safe: True

✓ All disk clusters are in database!

📊 82 clusters in database without disk directory (expected for some)

✅ MIGRATION COMPLETE AND SUCCESSFUL!
   - All cluster IDs are SQLite-safe
   - All disk directories match database


In [49]:
# Clean up any remaining oversized cluster directories
print("\n" + "="*60)
print("Cleanup Oversized Cluster Directories")
print("="*60)

oversized_dirs = []
if mmap_path.exists():
    for item in mmap_path.iterdir():
        if item.is_dir() and item.name.isdigit():
            try:
                cluster_id = int(item.name)
                if cluster_id >= 2**62:
                    oversized_dirs.append((cluster_id, item))
            except ValueError:
                pass

if oversized_dirs:
    print(f"\nFound {len(oversized_dirs)} oversized cluster directories to clean up:")
    
    for old_cid, old_path in oversized_dirs:
        print(f"\nProcessing: {old_cid}")
        
        # Find corresponding tiles in this directory
        tile_dirs = []
        year_dirs = []
        
        for item in old_path.iterdir():
            if item.is_dir() and item.name.isdigit() and len(item.name) == 4:
                year_dirs.append(item)
        
        if year_dirs:
            for year_dir in year_dirs:
                for item in year_dir.iterdir():
                    if item.is_dir() and '_' in item.name:
                        parts = item.name.split('_')
                        if len(parts) == 2:
                            try:
                                ix, iy = int(parts[0]), int(parts[1])
                                year = int(year_dir.name)
                                tile_dirs.append((ix, iy, year))
                                break
                            except ValueError:
                                pass
                if tile_dirs:
                    break
        
        if tile_dirs:
            ix, iy, year = tile_dirs[0]
            
            # Find the correct cluster ID in database
            cursor.execute("""
                SELECT DISTINCT cluster_id
                FROM tiles
                WHERE tile_ix = ? AND tile_iy = ? AND year = ?
                LIMIT 1
            """, (ix, iy, year))
            
            result = cursor.fetchone()
            
            if result:
                new_cid = result['cluster_id']
                new_path = mmap_path / str(new_cid)
                
                print(f"  Should be renamed to: {new_cid}")
                
                if new_path.exists():
                    print(f"  ⚠️  Destination already exists - these are duplicates")
                    print(f"  📁 Renaming old directory to backup: {old_cid}_backup")
                    backup_path = mmap_path / f"{old_cid}_backup"
                    shutil.move(str(old_path), str(backup_path))
                    print(f"  ✓ Moved to backup (can be manually merged/deleted)")
                else:
                    print(f"  Renaming: {old_cid} -> {new_cid}")
                    shutil.move(str(old_path), str(new_path))
                    print(f"  ✓ Renamed successfully")
            else:
                print(f"  ⚠️  No matching tiles found in database")
                print(f"  📁 Renaming to orphaned: {old_cid}_orphaned")
                orphaned_path = mmap_path / f"{old_cid}_orphaned"
                shutil.move(str(old_path), str(orphaned_path))
                print(f"  ✓ Moved to orphaned (can be manually reviewed)")
        else:
            print(f"  ⚠️  No tiles found in directory")
else:
    print("\n✓ No oversized cluster directories found")


Cleanup Oversized Cluster Directories

Found 1 oversized cluster directories to clean up:

Processing: 17162516979381748491
  Should be renamed to: 3327458924099584779
  ⚠️  Destination already exists - these are duplicates
  📁 Renaming old directory to backup: 17162516979381748491_backup
  ✓ Moved to backup (can be manually merged/deleted)


In [51]:
# Close database connection
conn.close()
print("✓ Database connection closed")

✓ Database connection closed
